In [1]:
import pickle as pickle
import os
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split

# Dataset 구성.
class RE_Dataset(torch.utils.data.Dataset):
  def __init__(self, tokenized_dataset, labels):
    self.tokenized_dataset = tokenized_dataset
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_dataset.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

# 처음 불러온 tsv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다.
# 변경한 DataFrame 형태는 baseline code description 이미지를 참고해주세요.
def preprocessing_dataset(dataset, label_type):
  label = []
  for i in dataset[8]:
    if i == 'blind':
      label.append(100)
    else:
      label.append(label_type[i])
  out_dataset = pd.DataFrame({'sentence':dataset[1],'entity_01':dataset[2],'entity_02':dataset[5],'label':label,})
  return out_dataset

# tsv 파일을 불러옵니다.
def load_data(dataset_dir):
  # load label_type, classes
  with open('/opt/ml/input/data/label_type.pkl', 'rb') as f:
    label_type = pickle.load(f)
  # load dataset
  dataset = pd.read_csv(dataset_dir, delimiter='\t', header=None)
  # preprecessing dataset
  dataset = preprocessing_dataset(dataset, label_type)
  
  return dataset

# bert input을 위한 tokenizing.
# tip! 다양한 종류의 tokenizer와 special token들을 활용하는 것으로도 새로운 시도를 해볼 수 있습니다.
# baseline code에서는 2가지 부분을 활용했습니다.
def tokenized_dataset(dataset, tokenizer):
  concat_entity = []
  for e01, e02 in zip(dataset['entity_01'], dataset['entity_02']):
    temp = ''
    temp = e01 + '[SEP]' + e02
    concat_entity.append(temp)
  tokenized_sentences = tokenizer(
      concat_entity,
      list(dataset['sentence']),
      return_tensors="pt",
      padding=True,
      truncation=True,
      max_length=100,
      add_special_tokens=True,
      )
  return tokenized_sentences


In [2]:
import pickle as pickle
import os
import pandas as pd
import torch
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, BertConfig
from transformers import ElectraForSequenceClassification, ElectraConfig, ElectraTokenizer
from transformers import XLMRobertaForSequenceClassification, XLMRobertaConfig, XLMRobertaTokenizer
from load_data import *
import wandb
import argparse
from sklearn.model_selection import train_test_split

In [3]:
MODEL_NAME2 = "monologg/koelectra-base-v3-discriminator"
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME2)

In [4]:
train_dataset = load_data("/opt/ml/input/data/train/train.tsv")
#dev_dataset = load_data("./dataset/train/dev.tsv")
train_label = train_dataset['label'].values

In [5]:
concat_entity = []
for e01, e02 in zip(train_dataset['entity_01'], train_dataset['entity_02']):
    temp = ''
    temp = e01 + '[SEP]' + e02
    concat_entity.append(temp)


In [9]:
sentences = list(train_dataset['sentence'])

In [111]:
def random_sentence_reverse(sentence, alpha=4, beta=10, prob=0.7):
    '''
    prob0 : 0 -> 바꾸지 않음, 1 -> 바꿈
    prob1 : 0 -> idx 기준 앞쪽 변경, 1 -> idx 기준 뒷쪽 변경
    prob2 : 뽑을 idx를 결정
    '''
    prob0 = np.random.choice(2,1, p=[prob, 1-prob])

    if prob0 == 0:
        return sentence
    splitted_sentence = sentence.split(' ')
    prob1 = np.random.choice(2, 1)
    prob2 = np.round(np.random.beta(a=4, b=10), 1)

    splitted_sentence_length = len(splitted_sentence)
    cut_idx = int(splitted_sentence_length*prob2)
    
    new_sentence = ''
    if prob1 == 0:
        subset = splitted_sentence[:cut_idx]
        subset = list(reversed(subset))
        splitted_sentence[:cut_idx] = subset
        new_sentence = ' '.join(splitted_sentence)
    
    else:
        subset = splitted_sentence[cut_idx:]
        subset = list(reversed(subset))
        splitted_sentence[cut_idx:] = subset
        new_sentence = ' '.join(splitted_sentence)

    return new_sentence
        


        
    




In [124]:
random_sentence_reverse(sentences[0])

'자동차의 유틸리티 스포츠 사용되는 영국에서 브랜드로는 랜드로버(Land Rover)와 지프(Jeep)가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다.'

In [113]:
np.random.choice(2,1)

array([0])

In [105]:
a = [1,2,3,4,5]
list(reversed(a))

[5, 4, 3, 2, 1]

In [14]:
prob = np.round(np.random.beta(a=4, b=10),1)

In [29]:
splitted_length = len(sentences[0].split(' ')) # 빈칸 기준으로 split한 list의 전체 길이

In [30]:
cut_idx = int(splitted_length*prob) # 이 인덱스까지 or 부터 자른다

In [31]:
int(temp)

9

In [28]:
sentences[0]

'영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)와 지프(Jeep)가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다.'

In [27]:
sentences[0].split(' ')[:int(temp)].

['영국에서',
 '사용되는',
 '스포츠',
 '유틸리티',
 '자동차의',
 '브랜드로는',
 '랜드로버(Land',
 'Rover)와',
 '지프(Jeep)가']

In [6]:
tokenized_sentences = tokenizer(
      concat_entity,
      list(train_dataset['sentence']),
      return_tensors="pt",
      padding=True,
      truncation=True,
      max_length=150,
      add_special_tokens=True,
      )

['[CLS]',
 '랜드',
 '##로',
 '##버',
 '[SEP]',
 '자동차',
 '[SEP]',
 '영국',
 '##에',
 '##서',
 '사용',
 '##되',
 '##는',
 '스포츠',
 '유틸리티',
 '자동차',
 '##의',
 '브랜드',
 '##로',
 '##는',
 '랜드',
 '##로',
 '##버',
 '(',
 'La',
 '##nd',
 'R',
 '##over',
 ')',
 '와',
 '지프',
 '(',
 'Je',
 '##ep',
 ')',
 '가',
 '있',
 '##으며',
 ',',
 '이',
 '브랜드',
 '##들',
 '##은',
 '자동차',
 '##의',
 '종류',
 '##를',
 '일컫',
 '##는',
 '말',
 '##로',
 '사용',
 '##되',
 '##기',
 '##도',
 '한다',
 '.',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD

In [38]:
tokenized_train = tokenized_dataset(train_dataset, tokenizer)

{'input_ids': tensor([[    2, 10086,  4239,  ...,     0,     0,     0],
        [    2,  6802,     3,  ...,     0,     0,     0],
        [    2,  6755,  7325,  ...,     0,     0,     0],
        ...,
        [    2,  2741,  4427,  ...,  4479,  4480,     3],
        [    2,  7020,  6819,  ...,     0,     0,     0],
        [    2,  3420, 20903,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [16]:
item = {key: torch.tensor(val[0]) for key, val in tokenized_train.items()}

ipykernel_launcher:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [29]:
len(train_dataset['sentence'][0])

96

In [30]:
item['input_ids'].

AttributeError: 'Tensor' object has no attribute 'tokens'

In [ ]:
RE_train_dataset = RE_Dataset(tokenized_train, train_label)

In [ ]:
def custom_aug(data):
    # randomly reverse order of sentence
    cut_idx = np.random.beta(a=5, b=1)
    



In [7]:
def random_sentence_reverse(sentence, alpha=5, beta=1):
    '''
    prob1 : 0 -> idx 기준 앞쪽 변경, 1 -> idx 기준 뒷쪽 변경
    prob2 : 뽑을 idx를 결정
    '''
    splitted_sentence = sentence.split(' ')
    prob1 = np.random.choice(2, 1)
    while True:
        prob2 = np.round(np.random.beta(a=4, b=10), 1)
        if prob2 != 0.0:
            break

    splitted_sentence_length = len(splitted_sentence)
    cut_idx = int(splitted_sentence_length*prob2)
    
    new_sentence = ''
    if prob1 == 0:
        subset = splitted_sentence[:cut_idx]
        subset = list(reversed(subset))
        splitted_sentence[:cut_idx] = subset
        new_sentence = ' '.join(splitted_sentence)
    
    else:
        subset = splitted_sentence[cut_idx:]
        subset = list(reversed(subset))
        splitted_sentence[cut_idx:] = subset
        new_sentence = ' '.join(splitted_sentence)

    return new_sentence

In [1]:
import pickle as pickle
import os
import pandas as pd
import torch
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, BertConfig
from transformers import ElectraForSequenceClassification, ElectraConfig, ElectraTokenizer
from transformers import XLMRobertaForSequenceClassification, XLMRobertaConfig, XLMRobertaTokenizer
from load_data import *
import wandb
import argparse
from sklearn.model_selection import train_test_split

In [2]:
train_dataset = pd.read_csv("/opt/ml/input/data/train/train.tsv", delimiter='\t', header=None)
#dev_dataset = load_data("./dataset/train/dev.tsv")
# train_label = train_dataset['label'].values

In [3]:
train_dataset.head()

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님


In [20]:
random_sentence_reverse(train_dataset.loc[0, 1])

'랜드로버(Land 브랜드로는 자동차의 유틸리티 스포츠 사용되는 영국에서 Rover)와 지프(Jeep)가 있으며, 이 브랜드들은 자동차의 종류를 일컫는 말로 사용되기도 한다.'

In [21]:
augmented_data = pd.DataFrame(columns=train_dataset.columns)

In [66]:
temp = train_dataset.loc[0]
temp[1] = random_sentence_reverse(temp[1])
augmented_data.append(temp)

ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 한다. 사용되기도 말로 일컫는 종류를 자...,랜드로버,30,33,자동차,19,21,단체:제작


In [22]:
augmented_data = pd.DataFrame(columns=train_dataset.columns)
for j in range(1,2):
    for i in range(len(train_dataset)):
        temp = train_dataset.loc[i]
        temp[0] = temp[0] + str(j)
        temp[1] = random_sentence_reverse(temp[1])
        augmented_data = augmented_data.append(temp) 

ipykernel_launcher:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
augmented_data.head()

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-211,스포츠 사용되는 영국에서 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-441,선거에서 민주당은 해산 전 의석인 230석에 그쳤다. 데 획득하는 30석)을 비례대...,민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-121,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-51,용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 성공했다. 데에 진출하는 플레이...,강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-401,퍼쿤 왕 수코타이의 사이 1247년 1237년에서 왕은 람캄행 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님


In [24]:
augmented_data = augmented_data.drop_duplicates(subset=[1, 2, 5])

In [25]:
len(augmented_data)

8996

In [26]:
augmented_data.to_csv("/opt/ml/input/data/train/train_augmented3_plain.tsv", sep='\t', header=None, index=False)

In [98]:
new_data = pd.concat([train_dataset, augmented_data], axis=0)

In [99]:
len(new_data)

45000

In [100]:
new_data.to_csv("/opt/ml/input/data/train/train_augmented2.tsv", sep='\t', header=None, index=False)

In [101]:
aug = pd.read_csv("/opt/ml/input/data/train/train_augmented2.tsv", delimiter='\t', header=None)
aug.head()

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님


In [102]:
len(aug)

45000

In [103]:
len(aug[1].unique())

36681

In [104]:
len(aug.drop_duplicates(subset=[1, 2, 5]))

37491

In [80]:
train = pd.read_csv("/opt/ml/input/data/train/train.tsv", delimiter='\t', header=None)

In [59]:
train.head()

,0,1,2,3,4,5,6,7,8
814,wikipedia-25120-28-13-15-18-20,건륭제의 치세 중기부터 강희제와 옹정제가 구축해 놓은 부를 지출하기 시작하여 북경의...,강희제,13,15,옹정제,18,20,관계_없음
1192,wikipedia-23541-3-30-31-46-47,장한을 항복하게 권한 것이 항우에게 공적으로 인정되어 항우가 제후들을 분봉할 때에 ...,항우,30,31,관중,46,47,관계_없음
8740,wikitree-23074-12-0-1-3-5,육군 예비역이 된 성규는 바쁘게 활동을 재개한다.,육군,0,1,예비역,3,5,단체:구성원
4258,wikipedia-4535-15-15-17-0-2,정명훈은 18세에 런던에서 줄리니가 연주하는 베토벤 교향곡 제7번을 듣고 작품의 핵...,줄리니,15,17,정명훈,0,2,관계_없음
1422,wikitree-765-11-4-6-35-37,"25일 광산구는 광주송정역 앞 건물, 구 청사를 비롯한 광산구 21개 동 행정복지센...",광산구,4,6,21개,35,37,관계_없음


In [81]:
additional = pd.read_csv(
            "/opt/ml/input/data/train/train_augmented3_plain.tsv", delimiter='\t', header=None)

In [68]:
additional.head()

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-211,스포츠 사용되는 영국에서 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-441,선거에서 민주당은 해산 전 의석인 230석에 그쳤다. 데 획득하는 30석)을 비례대...,민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-121,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-51,용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 성공했다. 데에 진출하는 플레이...,강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-401,퍼쿤 왕 수코타이의 사이 1247년 1237년에서 왕은 람캄행 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님


In [64]:
train_dataset.head()

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님


In [69]:
additional.loc[2,1] == train_dataset.loc[2,1]

True

In [35]:
additional_s = additional[1]
train_s = train[1]

KeyError: 8995

In [38]:
count

313

In [78]:
additional.reset_index()

,index,0,1,2,3,4,5,6,7,8
0,0,wikipedia-24896-25-30-33-19-211,스포츠 사용되는 영국에서 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,1,wikipedia-12728-224-5-7-42-441,선거에서 민주당은 해산 전 의석인 230석에 그쳤다. 데 획득하는 30석)을 비례대...,민주당,5,7,27석,42,44,관계_없음
2,3,wikipedia-11479-37-24-26-3-51,용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 성공했다. 데에 진출하는 플레이...,강수일,24,26,공격수,3,5,인물:직업/직함
3,4,wikipedia-15581-6-0-2-32-401,퍼쿤 왕 수코타이의 사이 1247년 1237년에서 왕은 람캄행 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님
4,5,wikitree-7590-2-33-35-19-211,대학 AFCD는 관련해 이와 소속 전문가와 세계동물보건기구(OIE)가 사람과 동물 ...,OIE,33,35,전문가,19,21,관계_없음
...,...,...,...,...,...,...,...,...,...,...
8990,8991,wikipedia-5414-12-15-21-0-41,독일은 1차전에서 사우디아라비아와의 월드컵 FIFA 2002년 8-0으로 승리하였는...,사우디아라비아,15,21,2002년,0,4,관계_없음
8991,8992,wikipedia-10384-4-12-14-0-11,시장 닛산은 토요타와 메이커인 2대 일본의 점유율을 높이기 위한 신차 개발을 계속하...,토요타,12,14,일본,0,1,단체:본사_국가
8992,8993,wikipedia-25913-6-8-10-93-1061,방호의의 손자 방덕룡(方德龍)은 1588년(선조 21년) 무과에 녹훈되었다. 2등에...,방덕룡,8,10,선무원종공신(宣武原從功臣),93,106,인물:직업/직함
8993,8994,wikitree-12062-15-0-3-46-471,시리즈를 그램’ ‘LG 2020년형 출시한 국내시장에 초 올해 LG전자는 이달부터 ...,LG전자,0,3,북미,46,47,관계_없음


In [79]:
additional.head()

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-211,스포츠 사용되는 영국에서 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-441,선거에서 민주당은 해산 전 의석인 230석에 그쳤다. 데 획득하는 30석)을 비례대...,민주당,5,7,27석,42,44,관계_없음
3,wikipedia-11479-37-24-26-3-51,용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 성공했다. 데에 진출하는 플레이...,강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-401,퍼쿤 왕 수코타이의 사이 1247년 1237년에서 왕은 람캄행 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님
5,wikitree-7590-2-33-35-19-211,대학 AFCD는 관련해 이와 소속 전문가와 세계동물보건기구(OIE)가 사람과 동물 ...,OIE,33,35,전문가,19,21,관계_없음
